<pre>
Copyright 2021 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# since both Jupyter and `isabelle-client` use `asyncio` we need to enable
# nested event loops. We don't need that when using `isabelle-client` from
# Python scripts outside Jupyter
import nest_asyncio

nest_asyncio.apply()

In [2]:
from isabelle_client import start_isabelle_server
# first, we start Isabelle server
server_info, _ = start_isabelle_server(
    name="test", port=9999, log_file="server.log"
)

In [3]:
import logging

from isabelle_client import get_isabelle_client
# then we create Python client to Isabelle server
isabelle = get_isabelle_client(server_info)
!rm -f session.log
# we will log all the messages from the server to a file
isabelle.logger = logging.getLogger()
isabelle.logger.setLevel(logging.INFO)
isabelle.logger.addHandler(logging.FileHandler("session.log"))

In [4]:
# suppose we have an Isabelle theory file
# (probably generated by another Python script)
!cat Example.thy

theory Example
imports Main
begin
lemma "\<forall> x. \<exists> y. x = y"
by auto
end

In [5]:
# we can send this file to the server and get a response
isabelle.use_theories(
    theories=["Example"], master_dir=".", watchdog_timeout=0
)

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"7e47533d-5470-4abd-b762-28ef6d8f3c08"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":99,"task":"7e47533d-5470-4abd-b762-28ef6d8f3c08","message":"theory Draft.Example 99%","kind":"writeln","session":"","theory":"Draft.Example"}', response_length=161),
 IsabelleResponse(response_type='NOTE', response_body='{"percentage":100,"task":"7e47533d-5470-4abd-b762-28ef6d8f3c08","message":"theory Draft.Example 100%","kind":"writeln","session":"","theory":"Draft.Example"}', response_length=163),
 IsabelleResponse(response_type='FINISHED', response_body='{"ok":true,"errors":[],"nodes":[{"messages":[],"exports":[],"status":{"percentage":100,"unprocessed":0,"running":0,"finished":7,"failed":0,"total":7,"consolidated":true,"canceled":false,"ok":true,"warn

In [6]:
# if we have a session description using ROOT file like this
!rm -rf output
!cat ROOT

session examples = HOL +
  options [document = pdf, document_output = "output"]
  theories
    Example
  document_files
    "root.tex"


In [7]:
# we can build session (`isabelle build`)
isabelle.session_build(dirs=["."], session="examples")

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='{"task":"fbb69167-f36b-4166-bad0-1a045b4ea361"}', response_length=None),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Building examples ...","task":"fbb69167-f36b-4166-bad0-1a045b4ea361"}', response_length=104),
 IsabelleResponse(response_type='NOTE', response_body='{"task":"fbb69167-f36b-4166-bad0-1a045b4ea361","message":"examples: theory examples.Example","kind":"writeln","session":"examples","theory":"examples.Example"}', response_length=165),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Preparing examples/document ...","task":"fbb69167-f36b-4166-bad0-1a045b4ea361"}', response_length=114),
 IsabelleResponse(response_type='NOTE', response_body='{"kind":"writeln","message":"Finished examples/document (0:00:01 elapsed

In [8]:
# the results will appear in `output` folder
# notice that `output` folder does not contain a PDF file
# because `build` command failed
# (this container has no LaTeX installed)
!ls output

document  document.pdf


In [9]:
from isabelle_client import async_run
# or we can issue a free-text command through TCP
# here `asynchronous` argument means 'asynchronous command' as defined
# in section 4.2.6. of Isabelle System manual. `echo` is a synchronous command
# but Python communicates with Isabelle asynchronously as always
async_run(isabelle.execute_command("echo 42", asynchronous=False))

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='42', response_length=None)]

In [10]:
# we can also shut down the Isabelle server
isabelle.shutdown()

[IsabelleResponse(response_type='OK', response_body='{"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}', response_length=None),
 IsabelleResponse(response_type='OK', response_body='', response_length=None)]

In [11]:
# all our communications with the server
# were written to the log file specified earlier
!cat session.log

8914c0bc-27e8-4578-9a16-1f00b13bb9ef
session_start {"session": "HOL"}

OK {"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}
OK {"task":"f0eff31e-a295-48bc-b2d2-b774f543276b"}
107
NOTE {"kind":"writeln","message":"Starting session HOL ...","task":"f0eff31e-a295-48bc-b2d2-b774f543276b"}
175
FINISHED {"session_id":"3783f4d0-1797-43fd-b972-b6fe847749fc","tmp_dir":"/tmp/isabelle-boris/server_session6048720414829119168","task":"f0eff31e-a295-48bc-b2d2-b774f543276b"}
8914c0bc-27e8-4578-9a16-1f00b13bb9ef
use_theories {"session_id": "3783f4d0-1797-43fd-b972-b6fe847749fc", "theories": ["Example"], "watchdog_timeout": 0, "master_dir": "."}

OK {"isabelle_id":"c2a2be496f35","isabelle_name":"Isabelle2021-1"}
OK {"task":"7e47533d-5470-4abd-b762-28ef6d8f3c08"}
161
NOTE {"percentage":99,"task":"7e47533d-5470-4abd-b762-28ef6d8f3c08","message":"theory Draft.Example 99%","kind":"writeln","session":"","theory":"Draft.Example"}
163
NOTE {"percentage":100,"task":"7e47533d-5470-4abd-b762-28ef6d